<a href="https://colab.research.google.com/github/Kevin-J-Klein/NGDS-Gruppe-G/blob/main/Copy_of_NN_projekt_GruppeM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip -q /content/fake-news.zip -d /content/fake_news

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd

In [ ]:
df_train = pd.read_csv("/content/fake_news/train.csv")
df_test = pd.read_csv("/content/fake_news/test.csv")
df_submit = pd.read_csv("/content/fake_news/submit.csv")

In [ ]:
# Check the shapes of the loaded DataFrames
print("Shape of train dataset:", df_train.shape)
print("Shape of test dataset:", df_test.shape)
print("Shape of Submit dataset", df_submit.shape)

Shape of train dataset: (20800, 5)
Shape of test dataset: (5200, 4)
Shape of Submit dataset (5200, 2)


In [ ]:
df_train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df_test.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [ ]:
df_submit.head()

,id,label
0,20800,0
1,20801,1
2,20802,0
3,20803,1
4,20804,1


In [ ]:
#DISTRIBUTION PLOT ??

PREPARE DATA
* Check for Null values and drop them
* Check for Duplicates and delet them

CLEAN TEXT DATA
* Conversion of the text to lower case.
*  Removing:
    * the HTML tags
    * the numbers
    * line breaks and tabs
    * quotation marks and hyphens
    * special quotation marks
    * URLs and repeated characters
    * punctuation
    * parts of text with several consecutive capital letters
    * the empty lines


In [ ]:
print("Data Information")
print(df_train.info())
print('\n', "+"*50 ,'\n')
print("Data Null Values")
print(df_train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB
None
id           0
title      558
author    1957
text        39
label        0
dtype: int64


In [ ]:
df_train = df_train.dropna()

In [ ]:
print(df_train.duplicated().sum())

0


In [ ]:
import re
import string
import warnings

# Funktion zur Textbereinigung definieren
def clean_text(text):
    text = text.lower()  # Kleinbuchstaben
    text = re.sub(r'<[^>]*>', '', text)  # HTML-Tags entfernen
    text = re.sub(r'\d+', '', text)  # Zahlen entfernen
    text = re.sub(r'[\n\t]', '', text)  # Zeilenumbrüche und Tabs entfernen
    text = re.sub(r'[\'\"\-]', '', text)  # Anführungszeichen und Bindestriche entfernen
    text = re.sub(r'[’‘“”]', '', text)  # Spezielle Anführungszeichen entfernen
    text = re.sub(r'(www\w*|http\w*|\\|\b\w*(\w)\2{5,}\w*\b)', '', text)  # URLs und wiederholte Zeichen entfernen
    text = ''.join([char for char in text if char not in string.punctuation])  # Interpunktion entfernen
    text = re.sub(r"[A-Z]{2,}", "", text)  # Überschriften entfernen
    return text.strip()  # Entfernt führende und folgende Leerzeichen

# Textbereinigung auf die Spalte 'text' anwenden
df_train['text'] = df_train['text'].apply(clean_text)

# Entfernen von leeren Zeilen
df_train = df_train[df_train['text'].str.strip().astype(bool)]

# Ergebnisse anzeigen
print(df_train)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [ ]:
# ist glaub unnötg???

import tensorflow as tf
# Konvertieren des DataFrames in ein tf.data.Dataset
def dataframe_to_dataset(df):
    df = df.copy()
    labels = df.pop('label')
    return tf.data.Dataset.from_tensor_slices((df['text'].values, labels.values))

# Erstellen des Datasets
tf_dataset = dataframe_to_dataset(df_train)

# Shufflen, Batchen und Vorbereitung für das Training
tf_dataset = tf_dataset.shuffle(buffer_size=len(df_train)).batch(32).prefetch(tf.data.AUTOTUNE)

# Anzeigen einiger Werte aus dem tf.data.Dataset
for text, label in tf_dataset.take(1):
    print(text.numpy(), label.numpy())

[b'\xd1\x81\xd1\x88\xd0\xb0 \xd1\x80\xd0\xb5\xd1\x84\xd0\xbe\xd1\x80\xd0\xbc\xd0\xb8\xd1\x80\xd0\xbe\xd0\xb2\xd0\xb0\xd1\x82\xd1\x8c\xd1\x81\xd1\x8f \xd0\xb8\xd0\xbb\xd0\xb8 \xd1\x80\xd0\xb0\xd1\x81\xd0\xbf\xd0\xb0\xd0\xb4\xd0\xb0\xd1\x82\xd1\x8c\xd1\x81\xd1\x8f \xd1\x82\xd1\x8c\xd0\xb5\xd1\x80\xd1\x80\xd0\xb8 \xd0\xbc\xd0\xb5\xd0\xb9\xd1\x81\xd0\xb0\xd0\xbd \xd0\xbd\xd0\xb0\xd0\xb1\xd0\xbb\xd1\x8e\xd0\xb4\xd0\xb0\xd1\x8f \xd0\xb7\xd0\xb0 \xd0\xbf\xd1\x80\xd0\xb5\xd0\xb7\xd0\xb8\xd0\xb4\xd0\xb5\xd0\xbd\xd1\x82\xd1\x81\xd0\xba\xd0\xbe\xd0\xb9 \xd0\xb3\xd0\xbe\xd0\xbd\xd0\xba\xd0\xbe\xd0\xb9 \xd0\xb2 \xd1\x81\xd1\x88\xd0\xb0 \xd1\x82\xd1\x8c\xd0\xb5\xd1\x80\xd1\x80\xd0\xb8 \xd0\xbc\xd0\xb5\xd0\xb9\xd1\x81\xd0\xb0\xd0\xbd \xd0\xbe\xd1\x82\xd0\xbc\xd0\xb5\xd1\x87\xd0\xb0\xd0\xb5\xd1\x82 \xd0\xb2\xd0\xbe\xd0\xb7\xd1\x80\xd0\xbe\xd0\xb6\xd0\xb4\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5 \xd0\xb4\xd0\xb0\xd0\xb2\xd0\xbd\xd0\xbe \xd0\xb7\xd0\xb0\xd0\xb1\xd1\x8b\xd1\x82\xd1\x8b\xd1\x85 \xd1\x80\xd0\xb0\xd

In [ ]:
# Funktion zur Textbearbeitung definieren
def shorten_long_texts(dataframe):
    # Durch die Zeilen des DataFrames iterieren
    for idx, text in dataframe['text'].iteritems():
        # Text in Wörter aufteilen
        words = text.split()
        # Überprüfen, ob der Text mehr als 100 Wörter hat
        if len(words) > 100:
            # Text auf die ersten 100 Wörter kürzen
            dataframe.at[idx, 'text'] = ' '.join(words[:100])
    return dataframe

# DataFrame bearbeiten
df5 = shorten_long_texts(df2)

# Ergebnisse anzeigen
print(df5)

TOKENIZE TEXT







